# Final Assignment

STT: 34

**Member**

18120374 Nguyễn Minh Hiếu


Thư viện cần dùng thêm:
   * **ntlk**: để schemming
   
   * **re** để làm chuẩn
   

In [ ]:
ntlk.download()
#Hiện bảng Chọn popular

# Cần làm xử lí để đọc 2 file `check.txt` và `stopword.txt` 

Lợi ích:
* Nếu tìm được công thức thì công ty sản xuất có thể tự động hóa quá trình rút trích ý kiến thích / không thích từ các đoạn đánh giá của người dùng
* Giúp công ty đánh giá được hiệu quả công việc. (Ở đây là AMAZON có thể kiểm tra sản phẩm được vận chuyển có đúng ý muốn khách hàng hay không)

# Bước 1: Thu thập dữ liệu
Đưa một link sản phẩm bất kì và kiểm tra phần đánh giá của khách hàng.
* Đánh giá sẽ phân loại: ***postive* : 1**  và ***critical* : 0**
* Mỗi trang gồm 10 đánh giá, ta duyệt 250 trang cho mỗi loại đánh giá. Vậy tổng cộng 10x270x2 = 5400 đánh giá

In [1]:
import requests
import pandas as pd
import numpy as np
import time
import sklearn
from bs4 import BeautifulSoup
from selenium import webdriver
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [2]:
url = 'https://www.amazon.com/NETGEAR-R6700-Nighthawk-Gigabit-Ethernet/dp/B00R2AZLD2/ref=gbps_img_m-9_475e_daf1a9de?smid=ATVPDKIKX0DER&pf_rd_p=5d86def2-ec10-4364-9008-8fbccf30475e&pf_rd_s=merchandised-search-9&pf_rd_t=101&pf_rd_i=15529609011&pf_rd_m=ATVPDKIKX0DER&pf_rd_r=B32H66M6KMBDBB378HQH&th=1'
browser = webdriver.Chrome(executable_path='./chromedriver.exe') 
browser.maximize_window()
browser.get(url)



`Phrases` Gồm các đánh giá

`status` loại đánh giá

In [ ]:
phrases = []
status = []

In [ ]:
#Click See all review
review_page = browser.find_element_by_link_text('See all reviews').click();

In [3]:
#Choose Verified purchase only
reviewer_types = browser.find_elements_by_css_selector('#reviewer-type-dropdown option')
for reviewer_type in reviewer_types:
    if "Verified purchase only" in reviewer_type.text:
        reviewer_type.click()
        break

In [4]:
#Positive first
comment_types = browser.find_elements_by_css_selector('#star-count-dropdown option')
for comment_type in comment_types:
    if "All positive" in comment_type.text:
        comment_type.click()
        break

In [6]:
#Page content
for i in range(0,3):
    time.sleep(3)
    html_text = browser.page_source
    tree = BeautifulSoup(html_text, 'html.parser')
    reviews = tree.find_all('div',{'data-hook':'review'})
    for review in reviews:
        content = review.find('a', {'data-hook': 'review-title'})
        phrases.append(content.text.strip())
        status.append(1)
    browser.find_element_by_class_name('a-last').click()
   

In [7]:
# then Critical
comment_types = browser.find_elements_by_css_selector('#star-count-dropdown option')
for comment_type in comment_types:
    if "All critical" in comment_type.text:
        comment_type.click()
        break

In [8]:
#Page content
for i in range(0,3):
    time.sleep(3)
    html_text = browser.page_source
    tree = BeautifulSoup(html_text, 'html.parser')
    reviews = tree.find_all('div',{'data-hook':'review'})
    for review in reviews:
        content = review.find('a', {'data-hook': 'review-title'})
        phrases.append(content.text.strip())
        status.append(0)
    browser.find_element_by_class_name('a-last').click()
    

In [9]:
Data = pd.DataFrame(
    {'Para': phrases,
     'Stt': status
    }
)

**Kiểm tra data**

Lưu data vào file "data.csv"

In [10]:
Data.shape

(5400, 2)

In [11]:
#Data.to_csv('data.csv',index = False,encoding='utf-16',sep = '\t')

* Sử dụng file data.csv có sẵn
* cột Para sẽ chứa các đánh giá
* cột status gồm loại đánh giá tương ứng (1: positive - 0: critical)

In [12]:
data = pd.read_csv('data.csv',encoding='utf-16',sep = '\t')
#data.head()
data[data.index == 2458]

,Para,Stt
2458,Great Product!,1


In [13]:
data.shape

(5400, 2)

Dữ liệu trùng lặp

In [14]:
data.duplicated().sum()

326

Loại bỏ trùng lặp 

In [15]:
data.drop_duplicates(subset=None, keep='first', inplace=True)
data.shape

(5074, 2)

Kiểm tra thiếu dữ liệu

In [16]:
data.isna().sum()

Para    0
Stt     0
dtype: int64

Kiểm tra độ cân bằng dữ liệu

In [17]:
data.Stt.mean()

0.48817500985415846

In [18]:
y_sr = data["Stt"] # sr là viết tắt của series
X_df = data.drop("Stt", axis=1)

In [19]:
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3, 
                                                              stratify=y_sr, random_state=0)

In [20]:
train_X_df.shape

(3551, 1)

In [21]:
train_y_sr.shape

(3551,)

# Bước 2 tiền xử lý dữ liệu

Ta cần thực hiện các bước: loại phần thừa, áp dụng *stemming* 
* Stemming: đưa các từ có hình thức tương đồng nhau về từ gốc (Ở đây ta chưa bàn về ngữ cảnh).
* Phần thừa như dấu câu, icon.

Vì ta sử dụng tiếng Anh nên các từ có dấu (khác a-z A-Z) cũng sẽ bị loại bỏ

In [32]:
import nltk
import re

#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.neural_network import MLPClassifier 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import  StandardScaler


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [23]:
#Steeming English only
englishStemmer=SnowballStemmer("english")



# đọc file check.txt
# đọc file stopword.txt

rồi xử lý trong hàm dưới

Hàm `Recontruct_Sentence` để xử lý câu:
   * Làm chuẩn câu
   * Tối giản từ
   * ghép các từ lại

In [24]:
def Recontruct_Sentence(sentence):
    sentence = re.sub('[^A-Za-z]',' ', sentence)
    sentence = sentence.lower()
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(englishStemmer.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

Class `trans_Sentence` sẽ chuyển DataFrame đầu vào thành DataFrame tối giản với phương thức *transform*.

In [25]:
#Test pipeline
class trans_Sentence(BaseEstimator, TransformerMixin):
    
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        # YOUR CODE HERE
        optimize= []
        self = X_df.copy()
        sentence_s = self.Para
        for sentence in sentence_s:
            optimize.append(Recontruct_Sentence(sentence))
        self.drop(['Para'],axis = 1)
        self['Para'] = optimize
        return self

TfidfVectorizer() tham khảo ở [link](https://thorpham.github.io/blog/2018/04/14/setmentation/) và [doc](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

Class `toarray_Tidf` gồm các tham số đầu vào dành cho TfidfVectorizer().
* fit: dùng `TfidfVectorizer()` fit tập dữ liệu đầu vào 
* transform dùng `TfidfVectorizer()` biến dữ liệu đầu vào thành mảng các vector dựa trên fit 

In [26]:
class toarray_Tidf(BaseEstimator, TransformerMixin):
    def __init__(self,max_features = None, ngram_range = None, stop_words = None):
        self.max_features = max_features
        self.ngram_range = ngram_range
        self.stop_words = stop_words 
        vect = TfidfVectorizer(max_features =  max_features, ngram_range =  ngram_range, stop_words =  self.stop_words)
        self.vect = vect
    def fit(self, X_df, y=None):
        self.vect.fit(X_df.Para)
        return self
    def transform(self, X_df, y=None):
        # YOUR CODE HERE
        return self.vect.transform(X_df.Para).toarray()

Xây dựng mạng Neural Network

In [27]:
neural = MLPClassifier(hidden_layer_sizes = 10, activation='tanh', solver='lbfgs', random_state=0, max_iter = 2750)
tidf = toarray_Tidf(stop_words = {'english'}, ngram_range = (1,1))
pipe_prog = make_pipeline(trans_Sentence(), tidf, neural)

pipe_prog

Pipeline(steps=[('trans_sentence', trans_Sentence()),
                ('toarray_tidf',
                 toarray_Tidf(ngram_range=(1, 1), stop_words={'english'})),
                ('mlpclassifier',
                 MLPClassifier(activation='tanh', hidden_layer_sizes=10,
                               max_iter=2750, random_state=0,
                               solver='lbfgs'))])

In [ ]:
num_max_features = [500,700,1100,None]
num_alpha = [0.1, 1, 10, 100, 1000]

In [28]:
train_errs = []
val_errs = []
best_val_err = float('inf')
best_alpha = None
best_best_feature = None
for max_features in num_max_features:
    for alpha in num_alpha:
        pipe_prog.set_params(toarray_tidf__max_features= max_features, mlpclassifier__alpha=alpha)
        let_fit = pipe_prog.fit(train_X_df,train_y_sr)
        
        error = (1- let_fit.score(train_X_df,train_y_sr))*100
        train_errs.append(error)
        
        error = (1 - let_fit.score(val_X_df,val_y_sr))*100
        val_errs.append(error)
        
        if (error < best_val_err):
            best_val_err = error
            best_alpha = alpha
            best_feature = max_features

trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)
trans (3551, 1)
trans (3551, 1)
trans (1523, 1)


In [29]:
best_val_err

15.101772816808934

In [30]:
train_y_sr

1374    1
2515    1
2525    1
2967    0
4210    0
       ..
4232    0
2041    1
3205    0
1389    1
1323    1
Name: Stt, Length: 3551, dtype: int64